In [1]:
from google.colab import drive # Підключаємо Диск
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Завантажуємо файли

import pandas as pd

file_path_ind = '/content/drive/MyDrive/Colab_Notebooks/industries.csv'
file_path_app = '/content/drive/MyDrive/Colab_Notebooks/applications.csv'
df_ind = pd.read_csv(file_path_ind)
df_app = pd.read_csv(file_path_app)

In [3]:
df_app = df_app.drop_duplicates(['applicant_id'], keep='last') # Прибери дублікати applicant_id

In [4]:
df_app=df_app.fillna({'External Rating': 0, 'Education level': 'Середня'}) # В полі 'External Rating' заповни відсутні значення нулями + В полі 'Education level' заповни відсутні значення текстом “Середня”

In [5]:
df = pd.merge(df_app, df_ind, on='Industry', how='left') # Додай до цього DataFrame дані з файлу industries.csv, а саме, рейтинги індустрій.

In [13]:
df = df.dropna(subset=['Amount']) # Рейтинг дорівнює нулю, якщо відсутнє значення 'Amount'
df = df_filtered = df.loc[df['External Rating'] != 0]  # або якщо 'External Rating' дорівнює нулю.
df['Age_Score'] = (df['Age'] >= 35) * 20 # Якщо вік заявника між 35 та 55, до рейтингу додається 20 балів
df['Marital bonus'] = (df['Marital status'] == 'Married') * 20 # Якщо заявника одружений, до рейтингу додається 20 балів
df['Location bonus'] = (df['Location'] == 'Київ чи область') * 10 # Якщо заявника знаходиться в Києві чи області, до рейтингу додається 10 балів
df['Ext_nom'] = (df['External Rating'] >= 7) * 20 # Якщо 'External Rating' більше чи дорівнює 7, до рейтингу додається 20 балів
df['Ext_denom'] = (df['External Rating'] <= 2) * -20 # Якщо 'External Rating' менше чи дорівнює 2, з рейтингу віднімається 20 балів

In [40]:
from datetime import datetime # Це щоб крутити датами

df['Applied at'] = pd.to_datetime(df['Applied at'], format='mixed') # Робимо дату із стрінгу
df['Weekday'] = df['Applied at'].dt.day_name() # Отримуємо назву дня тижня
df['NotWeekend'] = ((df['Weekday'] != 'Sunday') & (df['Weekday'] != 'Saturday')) * 20 # Якщо заявка була подана не у вихідні, до рейтингу додається 20 балів
df['Weeknum'] = df['Applied at'].dt.isocalendar().week # Формуємо номер тижня згідно ISO
df['Year'] = df['Applied at'].dt.isocalendar().year # Формуємо рік згідно ISO бо інакше порве перехідний тиждень
df['Year Week'] = df['Year'].astype(str) + '-' + df['Weeknum'].astype(str).str.zfill(2) # Перехідні тижні треба відсортувати згідно року. Я так думаю.

In [41]:
df['Final Score'] = df[['Score', 'Age_Score', 'Marital bonus', 'Location bonus', 'NotWeekend', 'Ext_nom', 'Ext_denom']].sum(axis=1) # Суммуємо все
df['Final Score'] = df['Final Score'].clip(upper=100) # Обрізаємо стелю до 100
df = df[df['Final Score'] >0] # прибираємо нульових

In [45]:
df_final=df.groupby('Year Week')['Final Score'].mean().apply(lambda x: f"{x:.2f}").reset_index() # Робимо розрахунок + гарний формат

In [46]:
print(df_final) # Наче все вірно.

  Year Week Final Score
0   2022-48       50.68
1   2022-49       49.47
2   2022-50       49.84
3   2022-51       46.98
4   2022-52       50.93
5   2023-01       51.17
6   2023-02       51.88
